# Video Undestanding

This Jupiter notebook contains the code to process a video using [Amazon Nova models](https://docs.aws.amazon.com/nova/) to [video understanding](https://docs.aws.amazon.com/nova/latest/userguide/modalities-video.html). If the video is less than 25MB, it is converted to base64, and if it's larger, it is uploaded to an Amazon S3 bucket, which must be added as a variable in **you_bucket**. 

![Diagram](data/video_understanding.png)






The Nova family introduces five specialized models tailored to different use cases:
- **Nova Pro:** A high-capability multimodal model balancing accuracy, speed, and cost
- **Nova Lite:** A faster, cost-effective multimodal model optimized for processing various media
types
- **Nova Micro:** A text-only model delivering low-latency responses at minimal cost
- **Nova Canvas:** An image generation model with professional-grade outputs and customization options
- **Nova Reel:** A video generation model with quality outputs and motion control features

> Using Nova pro and Nova Lite models you don't need to create a video embedding or store the video in a vector database, Amazon Nova does everything for you. Learn more about in [The Amazon Nova Family of Models:
Technical Report and Model Card](https://assets.amazon.science/10/0a/0b61d39a4e9aaec16f71ad3d9168/the-amazon-nova-family-of-models-technical-report-and-model-card2-26.pdf) and find more code samples in [aws-samples repo](https://github.com/aws-samples/amazon-nova-samples/)

![Diagram](./data/nova-models.png)

The credentials used to run this notebook requires permission for the *bedrock:InvokeModel* action to invoke Amazon Nova models and to [*PutObjet* to S3 bucket](https://docs.aws.amazon.com/AmazonS3/latest/API/API_PutObject.html). 

Additionally, remember to activate the model in the console; follow the [Getting started with Amazon Bedrock steps.](https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html)

## Requirements: 
- Install boto3 - This is the [AWS SDK for Python ](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingTheBotoAPI.html)that allows interacting with AWS services. Install with `pip install boto3`.
- [Configure AWS credentials](https://docs.aws.amazon.com/braket/latest/developerguide/braket-using-boto3.html) - Boto3 needs credentials to make API calls to AWS.


In [ ]:
#!pip install boto3
#!pip install langchain
#!pip install psycopg2-binary
#!pip install langchain_experimental
# or install requirements.txt

In [ ]:
import boto3 # to interact with AWS services.
import json
import base64
import os
from pathlib import Path

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"

In [ ]:
bedrock_client              = boto3.client("bedrock-runtime", region_name='us-east-1') 
boto3_bedrock               = boto3.client('bedrock', region_name='us-east-1')
s3_client                   = boto3.client('s3')
s3_uri = "s3://reinventagentstack-bucketagendabucket1c1c4a36-iiafx6tdvrak/video_demo/"

### Validate the connection
We can check the client works by trying out the list_foundation_models() method, which will tell us all the models available for us to use

In [ ]:
[models['modelId'] for models in boto3_bedrock.list_foundation_models()['modelSummaries']]

In [ ]:
def handle_video(video_path,prompt):
    # Convert 64MB to bytes
    SIZE_THRESHOLD = 64 * 1024 * 1024  # 64MB in bytes
    
    file_size = Path(video_path).stat().st_size
    file_name = os.path.basename(video_path)
    content_type = file_name.lower().split('.')[-1]
    
    try:
        if file_size <= SIZE_THRESHOLD:
            # For files under 64MB, create base64
            print(f"Video size: {file_size / (1024 * 1024):.2f} MB - Creating base64 - content type: {content_type}")
            with open(video_path, "rb") as file:
                media_bytes = file.read()
                

            messages = [
                        {
                            "role": "user",
                            "content": [
                                {"video": {"format": "mp4", "source": {"bytes": media_bytes}}},
                                {"text": prompt},
                            ],
                        }
                            ]
                
        else:
            # For files over 64MB, upload to S3
            print(f"Video size: {file_size / (1024 * 1024):.2f} MB - Uploading to S3")
            
            bucket_name = s3_uri.split('/')[2]
            media_uri = f'{s3_uri}/{file_name}'

            with open(video_path, "rb") as data:
                s3_client.upload_fileobj(data,bucket_name, media_uri)
            print("Put file in s3://{}{}{}".format(bucket_name,s3_uri,file_name))

            #put objet
            s3_client.put_object(
                Bucket=bucket_name,
                Key=file_name,
                Body=open(video_path, 'rb'),
                ContentType=f'video/{content_type}'
            )

            messages = [
                            {
                                "role": "user",
                                "content": [
                                    {
                                        "video": {
                                            "format": content_type,
                                            "source": {
                                                "s3Location": {
                                                    #Replace the s3 bucket URI 
                                                    "uri": media_uri
                                                }
                                            },
                                        }
                                    },
                                    {"text": prompt},
                                ],
                            }
                        ]

        inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

        model_response = bedrock_client.converse(
                modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
            )

        #print("\n[Full Response]")
        #print(json.dumps(model_response, indent=2))

        print("\n[Response Content Text]")
        response = model_response["output"]["message"]["content"][0]["text"]
        print(response)
    
            
    except Exception as e:
        print(f"Error processing video: {str(e)}")
        raise

      
    return response



In [ ]:
video_path = "demo-files/moderation-video.mp4"
prompt = "Describe the following video"
response = handle_video(video_path,prompt)

### How Amazon Nova Pro and Lite Understands Video 
When a video is submitted to Nova Pro or Lite, the system processes it as a sequence of frames sampled from the video. These frames are processed through the model’s vision encoders, which transform the visual information into embeddings that can be processed by the transformer architecture. 

For videos less than or equal to 16 minutes in duration, a 1 frame per second (FPS) sampling rate is employed.

However, for videos exceeding 16 minutes in length, the sampling rate decreases in order to maintain a consistent 960 frames sampled, with the frame sampling rate varying accordingly. 

This approach is designed to provide more accurate scene-level video understanding for shorter videos compared to longer video content. 

Learn more in [Video size information page](https://docs.aws.amazon.com/nova/latest/userguide/modalities-video.html#modalities-video-size)

> We recommend that you keep the video length less than 1 hour for low motion, and less than 16 minutes for anything with higher motion.
